** use object oriented programming: class **

always define a default function: init

In [15]:
class SimpleClass():
    # modify it after In [14]
    def __init__(self, name):
        print('hello ' + name)
    
    # added after In [7]
    def yell(self):
        print('YELLING')

In [2]:
s = 'world'

In [3]:
type(s)

str

In [4]:
x = SimpleClass

In [5]:
x

__main__.SimpleClass

let x be the instance of this class, use ()

In [6]:
x = SimpleClass()

hello


In [7]:
x

In [10]:
x2 = SimpleClass()

hello


In [11]:
x2.yell()

YELLING


extend a class from SimpleClass

SimpleClass's init will be overwritten by ExtendedClass

unless use super()

In [17]:
class ExtendedClass(SimpleClass):
    def __init__(self):
        super().__init__('Jose')
        print('EXTEND!')
        

In [13]:
y = ExtendedClass()

EXTEND!


In [14]:
y.yell()

YELLING


In [18]:
y2 = ExtendedClass()

hello Jose
EXTEND!


## operation

In [14]:
class Operation():
    # operation is a node in a Graph
    def __init__(self, input_nodes=[]):
        # store input nodes, init output nodes
        self.input_nodes = input_nodes
        self.output_nodes = []
        # for every input nodes, append this operation(self) to the output node
        for node in input_nodes:
            node.output_nodes.append(self)
        # a global default graph
        # append this operation to the list of operations in the current graph
        _default_graph.operations.append(self)
    
    def compute(self):
        # it will be overwritten, just use pass
        pass

In [15]:
# tensor flow use lower case for operation, so add, multiply ...
class add(Operation):
    def __init__(self, x, y):
        # overwrite __init__
        super().__init__([x, y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [16]:
class multiply(Operation):
    def __init__(self, x, y):
        # overwrite __init__
        super().__init__([x, y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [17]:
class matmul(Operation):
    def __init__(self, x, y):
        # overwrite __init__
        super().__init__([x, y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

In [18]:
class Placeholder():
    def __init__(self):
        self.output_nodes = []
        _default_graph.placeholders.append(self)

In [19]:
class Variable():
    def __init__(self, initial_value = None):
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [20]:
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
    
    def set_as_default(self):
        global _default_graph
        _default_graph = self

z = Ax + b

A = 10

b = 1

z = 10x + 1

In [21]:
g = Graph()

In [22]:
g.set_as_default()

In [23]:
A = Variable(10)

In [24]:
b = Variable(1)

In [25]:
x = Placeholder()

In [26]:
y = multiply(A, x)

In [27]:
z = add(y, b)

In [28]:
import numpy as np

def traverse_postorder(operation):
    '''
    PostOrder Traversal of Nodes. Basically makes sure computations are
    done in the correct order (Ax first, then Ax + b). Feel free to copy
    and paste this. It is not super important for understanding the basic
    fundamentals of deep learning
    '''
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
    
    recurse(operation)
    return nodes_postorder

In [29]:
class Session:
    #operation to compute ;  feed_dict: mapping placeholder to input values
    # feed batches of data
    def run(self, operation, feed_dict = {}):
        nodes_postorder = traverse_postorder(operation)
        for node in nodes_postorder:
            if type(node) == Placeholder:
                node.output = feed_dict[node] # dictionary value
            elif type(node) == Variable:
                node.output = node.value
            else:
                # if not placeholder and variable, it must be operation
                node.inputs = [input_node.output for input_node in node.input_nodes]
                node.output = node.compute(*node.inputs)  # * for args
            
            if type(node.output) == list:
                node.output = np.array(node.output)  # conver to np array
        return operation.output

In [35]:
sess = Session()

In [36]:
result = sess.run(operation=z,feed_dict={x:10})

In [37]:
result

101